# Mount the Gdrive


In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!ls "/content/gdrive/My Drive/eva4p2/drone-dataset1.zip"

'/content/gdrive/My Drive/eva4p2/drone-dataset1.zip'


In [3]:
cp -r "/content/gdrive/My Drive/eva4p2/drone-dataset1.zip" "drone-dataset.zip"

In [4]:
!unzip drone-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: drone-dataset/train/3/Image_67.jpg  
  inflating: drone-dataset/train/3/sandhu-eva4p2-imgg0174.jpg  
  inflating: drone-dataset/train/3/wing_Drone_0_B _95_.jpg  
  inflating: drone-dataset/train/3/capture1.jpg  
  inflating: drone-dataset/train/3/dsc07909.jpg  
  inflating: drone-dataset/train/3/image221.jpg  
  inflating: drone-dataset/train/3/217095_web.jpg  
  inflating: drone-dataset/train/3/5e31b98be0.jpg  
  inflating: drone-dataset/train/3/Image_316.jpg  
  inflating: drone-dataset/train/3/Image_417.jpg  
  inflating: drone-dataset/train/3/Image_692.jpg  
  inflating: drone-dataset/train/3/Tellus4D-2.jpg  
  inflating: drone-dataset/train/3/Image_144.jpg  
  inflating: drone-dataset/train/3/Image_293.jpg  
  inflating: drone-dataset/train/3/SkyWalker-X8-Heliguy-crop(1).jpg  
  inflating: drone-dataset/train/3/685080fc37bf3b1a28f92578413146d5.jpg  
  inflating: drone-dataset/train/3/dt26-open-payload-package.jpg  
  

#Custom Dataset

In [6]:
import numpy as np
import cv2
import io
import os
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor
from torchvision import transforms
import zipfile
from PIL import Image
import sys
import torch.utils.data
import timeit
import logging

# This is the common class for both test and train data sets. Based on indexes, this class serves the transformed image tensor.
  
#Considered 4 folders (n). Take all the files from those folders into a list, a global list in memory.     
#Flying birds, Large Quadcopters, Small QuadCopters, Winged drones
#Given an index, list[index] will be returned as a file.
#All images are of jpg and 224*224 size.

class WingsDataset(Dataset):

  def __init__(self, root, transform=ToTensor()):
    'Initialization'
    self.m_root = root
    self.m_transform = transform
    logging.basicConfig(filename = "/content/dataset_train_test_timeit.log",datefmt='%H:%M:%S',level=logging.DEBUG)
    #self.g_dict_fileindex = dict()
    self.g_array_fileindex = []
    self.g_array_targetindex = []
    self.walkTheDirectory(self.m_root) 
    self.m_length = len(self.g_array_fileindex)
  
  #root_dir structure
  #train
  #   /folder1
  #     file.jpg
  #     file40.jpg  
  #   /folder2
  #     filex.jpg
  #     filey.40.jpg  
  #   /folder3
  #     filea.jpg
  #     fileb.40.jpg  
  #   /folder4
  #     fileaa.jpg
  #     filebax.jpg  
  
  #test
  #   /anyfoldername1
  #     sfile.jpg
  #     mfile40.jpg  
  #   /foldername2
  #     efilex.jpg
  #     cfiley.40.jpg  
  #   /foldername3
  #     somefilea.jpg
  #     vfileb.40.jpg  
  #   /foldername4
  #     somefileaa.jpg
  #     afilebax.jpg  
  
  
    
  def walkTheDirectory(self,root_dir):
    #for root, dirs, files in os.walk('python/Lib/email'):
    # for file in files:
    #    with open(os.path.join(root, file), "r") as auto:    
    for dir_path, subdirs, files in os.walk(self.m_root): # path should give the complete path till the last directory.
        for filename in files:
            #print(os.path.join(path, filename))
            full_file_path = os.path.join(dir_path, filename)
            #self.g_dict_fileindex[index] = full_path
            self.g_array_fileindex.append(full_file_path)
            dir_name = os.path.basename(dir_path)#get the last directory name from full dir path
            self.g_array_targetindex.append(int(dir_name))

  def __len__(self):
    'Denotes the total number of samples'
    return self.m_length;

  def printItem(self,idx):
    if idx >= self.m_length:
        print('index', idx, 'is bigger than the actual dataset size')
        return
    file = self.g_array_fileindex[idx]
    print('printItem-',file)  

  def __getitem__(self, idx):

    if idx >= self.m_length:
        print('index', idx, 'is bigger than the actual dataset size')
        return
        
    start_time = timeit.default_timer()

    # Pick the file from the index
    filepath = self.g_array_fileindex[idx]
    target = self.g_array_targetindex[idx]

    #already of 224*224 , so dont do any resizing..
    #Apply transformation and give the tensor back
    image = Image.open(filepath).convert('RGB') # even grey scale images will be converted to RGB with 3 channels
    """
    w,h = image.size
    image = image.resize((224,224)) # These are already resized images. Dont do preprocessing here..
    """

    if self.m_transform:
      image = self.m_transform(image)
    
    load_time = timeit.default_timer() - start_time
    #target = torch.Tensor(int(target))
    desc = f' file={filepath} target={target} LOAD_TIME={load_time:0.3f}'
    logging.info(desc)
    return image, target
    

#Train/Test loader

In [7]:
import torchvision
import torch
import torchvision.transforms as transforms

def load_dataset():
    train_data_path = '/content/drone-dataset/train'
    test_data_path = '/content/drone-dataset/test'
    #train_dataset = torchvision.datasets.ImageFolder(
    train_dataset = WingsDataset(    
        root=train_data_path,
        transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.Grayscale(num_output_channels=3),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=16,
        num_workers=0,
        shuffle=True
    )
    
    #test_dataset = torchvision.datasets.ImageFolder(
    test_dataset = WingsDataset(        
        root=test_data_path,
        transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=16,
        num_workers=0,
        shuffle=True
    )
    return train_loader, test_loader

train_loader, test_loader = load_dataset()

#Model Used

In [18]:
import torch
import torch.nn.functional as F
import torch.nn as nn

class DroneMobilenetV2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Use ROIpoolng to resize image to 224*224
        #self.ROIpool = nn.AdaptiveMaxPool2d((224,224))
        # Use a pretrained MobilenetV2 model
        self.network = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=True)
        # Replace last layer
        num_ftrs = self.network.classifier[1].in_features
        #Add fully connected layer
        self.network.classifier[1] = nn.Linear(num_ftrs, 4) # output 4 classifications
    
    def forward(self, xb):
        #resized = self.ROIpool(xb)
        out     = self.network(xb)
        return F.log_softmax(out)
#         return torch.sigmoid(self.fc(self.relu(self.network(xb))))

    
    def freeze(self):
        # To freeze the residual layers
        for param in self.network.parameters():
            param.require_grad = False
        for param in self.network.classifier[1].parameters():
            param.require_grad = True
    
    def unfreeze(self):
        # Unfreeze all layers
        for param in self.network.parameters():
            param.require_grad = True

#Show Summary

In [19]:
import torch
import torch.optim as optim
from torchsummary import summary

model = DroneMobilenetV2().cuda()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

summary(model, input_size=(3,224,224))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
  InvertedResidual-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 96, 112, 112]           1,536
      BatchNorm2d-11         [-1, 96, 112, 112]             192
            ReLU6-12         [-1, 96, 112, 112]               0
           Conv2d-13           [-1, 96, 56, 56]             864
      BatchNorm2d-14           [-1, 96,

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


#Freeze the Model

In [21]:

model.freeze()

# Train, Test utility functions

In [20]:
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim

train_losses = []
train_acc = []

def train(model, device, trainloader, epoch):
    running_loss = 0.00
    correct = 0.0
    processed = 0
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    pbar = tqdm(trainloader)
    for i, data in enumerate(pbar):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        train_losses.append(loss)
        loss.backward()
        optimizer.step()
        output = outputs.argmax(dim=1, keepdim=True)
        correct = correct + output.eq(labels.view_as(output)).sum().item()
        processed = processed + len(inputs)
        pbar.set_description(desc= f'Loss={loss.item()} Accuracy={100*correct/processed:.2f}')
        train_acc.append(100*correct/processed)

import torch
import torch.nn.functional as F

test_losses = []
test_acc = []

def test(model, device, testloader):
    correct = 0
    total = 0
    test_loss = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            test_loss += F.nll_loss(outputs, labels, reduction='sum').item()
            #criterion = nn.CrossEntropyLoss()
            #test_loss += criterion(outputs, labels, reduction='sum').item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    test_losses.append(test_loss)
    print('\nTest Set: Average loss: {}, Accuracy: {}/{} ({:.2f}%)\n'.format(test_loss, correct, len(testloader.dataset),
                                                                        100. * correct / len(testloader.dataset)))


# Train the data

In [ ]:
for epoch in range(5):
    train(model, device, train_loader, epoch)
    test(model, device, test_loader)
    torch.save(model, 'drone_mobilenet_v2_3.pt')
    print("*********** Epoch {} ended *************".format(epoch))

Loss=0.12595342099666595 Accuracy=93.75:   0%|          | 1/707 [00:00<01:29,  7.89it/s]


Test Set: Average loss: 1746.4135016202927, Accuracy: 4240/4826 (87.86%)

*********** Epoch 0 ended *************


  0%|          | 0/707 [00:00<?, ?it/s]


Test Set: Average loss: 1606.6312779337168, Accuracy: 4334/4826 (89.81%)

*********** Epoch 1 ended *************


  0%|          | 0/707 [00:00<?, ?it/s]


Test Set: Average loss: 1508.889540374279, Accuracy: 4318/4826 (89.47%)

*********** Epoch 2 ended *************


  0%|          | 0/707 [00:00<?, ?it/s]


Test Set: Average loss: 1402.6506829410791, Accuracy: 4356/4826 (90.26%)

*********** Epoch 3 ended *************


Loss=7.402228832244873 Accuracy=96.52: 100%|██████████| 707/707 [04:14<00:00,  2.77it/s]



Test Set: Average loss: 1462.7111346125603, Accuracy: 4418/4826 (91.55%)

*********** Epoch 4 ended *************


#Using Customdataset loaders

In [12]:
for epoch in range(5):
    train(model, device, train_loader, epoch)
    test(model, device, test_loader)
    torch.save(model, 'drone_mobilenet_v2_3.pt')
    print("*********** Epoch {} ended *************".format(epoch))

  0%|          | 0/707 [00:00<?, ?it/s]


Test Set: Average loss: 1813.9237259030342, Accuracy: 4207/4827 (87.16%)

*********** Epoch 0 ended *************


Loss=0.09259136766195297 Accuracy=93.75:   0%|          | 1/707 [00:00<01:29,  7.85it/s]


Test Set: Average loss: 1614.1679657697678, Accuracy: 4321/4827 (89.52%)

*********** Epoch 1 ended *************


  0%|          | 0/707 [00:00<?, ?it/s]


Test Set: Average loss: 1403.5720526576042, Accuracy: 4369/4827 (90.51%)

*********** Epoch 2 ended *************


  0%|          | 0/707 [00:00<?, ?it/s]


Test Set: Average loss: 1574.5270193070173, Accuracy: 4370/4827 (90.53%)

*********** Epoch 3 ended *************


Loss=0.012967996299266815 Accuracy=96.42: 100%|██████████| 707/707 [04:02<00:00,  2.91it/s]



Test Set: Average loss: 1348.7484865039587, Accuracy: 4410/4827 (91.36%)

*********** Epoch 4 ended *************


#Train data with PIL resized images

In [14]:
cp -r "/content/gdrive/My Drive/processed-drone-dataset.zip" "processed-drone-dataset.zip"

In [15]:
!unzip processed-drone-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: content/processed-drone-dataset/train/1/Capture161.jpg  
  inflating: content/processed-drone-dataset/train/1/i (14).jpg  
  inflating: content/processed-drone-dataset/train/1/Capture170.jpg  
  inflating: content/processed-drone-dataset/train/1/i (15)-1.jpg  
  inflating: content/processed-drone-dataset/train/1/photo-1470083010285-12169927e64c.jpg  
  inflating: content/processed-drone-dataset/train/1/photo-1488263590619-bc1fff43b6c1.jpg  
  inflating: content/processed-drone-dataset/train/1/Screen-Shot-2017-10-24-at-2.29.18-PM-e1508869812636.jpg  
  inflating: content/processed-drone-dataset/train/1/pexels-photo-1379371.jpg  
  inflating: content/processed-drone-dataset/train/1/Capture14.jpg  
  inflating: content/processed-drone-dataset/train/1/pexels-photo-1852984.jpg  
  inflating: content/processed-drone-dataset/train/1/Capture139.jpg  
  inflating: content/processed-drone-dataset/train/1/Capture149.jpg  
  inflating

#Use PIL resized images

In [22]:
import torchvision
import torch
import torchvision.transforms as transforms

def load_dataset():
    train_data_path = '/content/content/processed-drone-dataset/train'
    test_data_path = '/content/content/processed-drone-dataset/test'
    #train_dataset = torchvision.datasets.ImageFolder(
    train_dataset = WingsDataset(    
        root=train_data_path,
        transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.Grayscale(num_output_channels=3),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        #transforms.Normalize([0.5271, 0.5788, 0.6095], [0.1707, 0.1650, 0.1804])#mean std of resized images
    ])
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=16,
        num_workers=0,
        shuffle=True
    )
    
    #test_dataset = torchvision.datasets.ImageFolder(
    test_dataset = WingsDataset(        
        root=test_data_path,
        transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=16,
        num_workers=0,
        shuffle=True
    )
    return train_loader, test_loader

train_loader, test_loader = load_dataset()

In [23]:
for epoch in range(5):
    train(model, device, train_loader, epoch)
    test(model, device, test_loader)
    torch.save(model, 'drone_mobilenet_v2_3.pt')
    print("*********** Epoch {} ended *************".format(epoch))


  0%|          | 0/707 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.

Loss=0.08132439106702805 Accuracy=88.72: 100%|██████████| 707/707 [00:58<00:00, 12.17it/s]

Loss=0.1710856705904007 Accuracy=93.75:   0%|          | 0/707 [00:00<?, ?it/s]



Test Set: Average loss: 1746.8565632104874, Accuracy: 4183/4827 (86.66%)

*********** Epoch 0 ended *************


Loss=0.4366130828857422 Accuracy=92.36: 100%|██████████| 707/707 [00:57<00:00, 12.20it/s]

Loss=0.041292306035757065 Accuracy=100.00:   0%|          | 0/707 [00:00<?, ?it/s]


Test Set: Average loss: 1555.987327247858, Accuracy: 4274/4827 (88.54%)

*********** Epoch 1 ended *************



Loss=0.04832548275589943 Accuracy=94.60: 100%|██████████| 707/707 [00:58<00:00, 12.16it/s]

Loss=0.10348451882600784 Accuracy=93.75:   0%|          | 0/707 [00:00<?, ?it/s]


Test Set: Average loss: 1505.5632030367851, Accuracy: 4309/4827 (89.27%)

*********** Epoch 2 ended *************



Loss=0.3432321846485138 Accuracy=94.83: 100%|██████████| 707/707 [00:58<00:00, 12.13it/s]

Loss=0.3845587372779846 Accuracy=81.25:   0%|          | 0/707 [00:00<?, ?it/s]


Test Set: Average loss: 1498.1513252705336, Accuracy: 4321/4827 (89.52%)

*********** Epoch 3 ended *************



Loss=4.666780948638916 Accuracy=95.90: 100%|██████████| 707/707 [00:58<00:00, 12.13it/s]



Test Set: Average loss: 1502.9132248088717, Accuracy: 4348/4827 (90.08%)

*********** Epoch 4 ended *************


###The below content is a copy paste about mobilenet information..
### This notebook is optionally accelerated with a GPU runtime.
### If you would like to use this acceleration, please select the menu option "Runtime" -> "Change runtime type", select "Hardware Accelerator" -> "GPU" and click "SAVE"

----------------------------------------------------------------------

# MobileNet v2

*Author: Pytorch Team*

**Efficient networks optimized for speed and memory, with residual blocks**

_ | _
- | -
![alt](https://pytorch.org/assets/images/mobilenet_v2_1.png) | ![alt](https://pytorch.org/assets/images/mobilenet_v2_2.png)

In [ ]:
import torch
#define the model
model = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=True)
model.eval()

In [ ]:
#trace model with a dummy input
traced_model = torch.jit.trace(model, torch.randn(1,3,224,224))
traced_model.save('mobilenetV2.pt')

All pre-trained models expect input images normalized in the same way,
i.e. mini-batches of 3-channel RGB images of shape `(3 x H x W)`, where `H` and `W` are expected to be at least `224`.
The images have to be loaded in to a range of `[0, 1]` and then normalized using `mean = [0.485, 0.456, 0.406]`
and `std = [0.229, 0.224, 0.225]`.

Here's a sample execution.

In [ ]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [ ]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
print(torch.nn.functional.softmax(output[0], dim=0))


### Model Description

The MobileNet v2 architecture is based on an inverted residual structure where the input and output of the residual block are thin bottleneck layers opposite to traditional residual models which use expanded representations in the input. MobileNet v2 uses lightweight depthwise convolutions to filter features in the intermediate expansion layer. Additionally, non-linearities in the narrow layers were removed in order to maintain representational power.

| Model structure | Top-1 error | Top-5 error |
| --------------- | ----------- | ----------- |
|  mobilenet_v2       | 28.12       | 9.71       |


### References

 - [MobileNetV2: Inverted Residuals and Linear Bottlenecks](https://arxiv.org/abs/1801.04381)